In [1]:
# This data handling code is adapted from the PyTorch geometric collection of google colab notebooks, a fantastic resource for getting started with GNNs. https://pytorch-geometric.readthedocs.io/en/latest/notes/colabs.html
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import DataLoader
# import the graph classifier you built in the last step
from GCN_03 import NodeClassifier, NodeClassifierWelling

/Users/default/miniconda3/envs/deep_learning_2023/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# - - - DATA PREPARATIONS - - -

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')
data = dataset[0]  # Get the first graph object.
print()
print(data)
print('=============================================================')
# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


/Users/default/miniconda3/envs/deep_learning_2023/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)
/Users/default/miniconda3/envs/deep_learning_2023/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'contains_self_loops' is deprecated, use 'has_self_loops' instead
  warnings.warn(out)


The size of tensor a (2708) must match the size of tensor b (13264) at non-singleton dimension 0

In [3]:
def train(model):
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask],
                     data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss


In [4]:
def test(model):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
    return test_acc

New Model 

In [5]:
model_new = NodeClassifier(num_node_features=1433, hidden_features=16, num_classes=7)
optimizer = torch.optim.Adam(model_new.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [6]:
for epoch in range(1, 201):
    loss = train(model_new)
    if epoch % 10 == 0:
        test_acc = test(model_new)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Test Accuracy: {test_acc}')

Epoch: 010, Loss: 1.8473, Test Accuracy: 0.26
Epoch: 020, Loss: 1.5505, Test Accuracy: 0.427
Epoch: 030, Loss: 1.1373, Test Accuracy: 0.42
Epoch: 040, Loss: 0.9260, Test Accuracy: 0.516
Epoch: 050, Loss: 0.6530, Test Accuracy: 0.572
Epoch: 060, Loss: 0.4179, Test Accuracy: 0.664
Epoch: 070, Loss: 0.5334, Test Accuracy: 0.625
Epoch: 080, Loss: 0.3605, Test Accuracy: 0.681
Epoch: 090, Loss: 0.2012, Test Accuracy: 0.67
Epoch: 100, Loss: 0.1835, Test Accuracy: 0.672
Epoch: 110, Loss: 0.1950, Test Accuracy: 0.643
Epoch: 120, Loss: 0.1559, Test Accuracy: 0.689
Epoch: 130, Loss: 0.1862, Test Accuracy: 0.697
Epoch: 140, Loss: 0.0799, Test Accuracy: 0.665
Epoch: 150, Loss: 0.1277, Test Accuracy: 0.683
Epoch: 160, Loss: 0.0601, Test Accuracy: 0.677
Epoch: 170, Loss: 0.0823, Test Accuracy: 0.679
Epoch: 180, Loss: 0.0749, Test Accuracy: 0.658
Epoch: 190, Loss: 0.0915, Test Accuracy: 0.673
Epoch: 200, Loss: 0.0520, Test Accuracy: 0.647


Old model (Welling et al., 2011) for node classification

In [11]:
model_welling = NodeClassifierWelling(num_node_features=1433, hidden_features=16, num_classes=7)
optimizer = torch.optim.Adam(model_welling.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [12]:
for epoch in range(1, 201):
    loss = train(model_welling)
    if epoch % 10 == 0:
        test_acc = test(model_welling)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Test Accuracy: {test_acc}')

Epoch: 010, Loss: 1.8950, Test Accuracy: 0.378
Epoch: 020, Loss: 1.7912, Test Accuracy: 0.333
Epoch: 030, Loss: 1.6343, Test Accuracy: 0.447
Epoch: 040, Loss: 1.4142, Test Accuracy: 0.553
Epoch: 050, Loss: 1.2101, Test Accuracy: 0.654
Epoch: 060, Loss: 0.9750, Test Accuracy: 0.727
Epoch: 070, Loss: 0.7839, Test Accuracy: 0.742
Epoch: 080, Loss: 0.6451, Test Accuracy: 0.754
Epoch: 090, Loss: 0.5383, Test Accuracy: 0.761
Epoch: 100, Loss: 0.3967, Test Accuracy: 0.756
Epoch: 110, Loss: 0.3238, Test Accuracy: 0.778
Epoch: 120, Loss: 0.2910, Test Accuracy: 0.773
Epoch: 130, Loss: 0.2634, Test Accuracy: 0.771
Epoch: 140, Loss: 0.2391, Test Accuracy: 0.787
Epoch: 150, Loss: 0.2010, Test Accuracy: 0.777
Epoch: 160, Loss: 0.1548, Test Accuracy: 0.784
Epoch: 170, Loss: 0.1632, Test Accuracy: 0.77
Epoch: 180, Loss: 0.1594, Test Accuracy: 0.78
Epoch: 190, Loss: 0.1382, Test Accuracy: 0.778
Epoch: 200, Loss: 0.1668, Test Accuracy: 0.779


The new model model has lower loss, but also lower accuracy. 